# Аналитика продаж для сети видеопроката

---

## Цель исследования:

Проанализировать продажи для сети видеопроката:
- [Провести предобработку данных](#clean);
- Провести исследовательский анализ данных;
- Сформулировать и проверить статистические гипотезы:
  - *Гипотеза различия среднемесячной выручки для жанров «Action» и «Comedy»*;
  - *Гипотеза различия общей выручки клиентов сегментов High value и Medium value*;

## Входные данные:

Входные данные представляют собой **4 датасета** (файлы *customer_ltv.csv*, *rental_activity_by_weekday_and_hour.csv*, *revenue_by_category.csv*, *revenue_by_genre_per_month.csv*), содержащие агрегированные данные о выручке по жанрам и категориям, активности аренды по времени, а также LTV клиентов, подготовленные на основе транзакций видеопроката.

### Предобработка данных <a id="clean"></a>

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Настройки отображения
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")
sns.set(style="whitegrid", palette="muted")

In [2]:
def load_csv(path):
    return pd.read_csv(path)

def convert_date_columns(df, *args):
    df = df.copy()
    for col in args:
        try:
            df[col] = pd.to_datetime(df[col])
        except:
            pass
    return df

In [3]:
datasets = {
    "revenue_by_genre_per_month": "./csv/revenue_by_genre_per_month.csv",
    "rental_activity_by_weekday_and_hour": "./csv/rental_activity_by_weekday_and_hour.csv",
    "revenue_by_category": "./csv/revenue_by_category.csv",
    "customer_ltv": "./csv/customer_ltv.csv"
}

dfs = {}
for name, path in datasets.items():
    try:
        df = pd.read_csv(path)
        dfs[name] = df
        print(f"Загружено: {name:35s}")
    except FileNotFoundError:
        print(f"Файл не найден: {path}")

Загружено: revenue_by_genre_per_month         
Загружено: rental_activity_by_weekday_and_hour
Загружено: revenue_by_category                
Загружено: customer_ltv                       


In [4]:
dfs["revenue_by_genre_per_month"] = convert_date_columns(dfs["revenue_by_genre_per_month"], 'month')
dfs["customer_ltv"] = convert_date_columns(dfs["customer_ltv"], 'create_date', 'first_rental_date', 'last_rental_date')

In [5]:
weekday_map = {0: "Sunday", 1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday"}
dfs["rental_activity_by_weekday_and_hour"]["weekday"] = dfs["rental_activity_by_weekday_and_hour"]["weekday"].map(weekday_map)

cols = {
    "revenue_by_genre_per_month": "category",
    "rental_activity_by_weekday_and_hour": "weekday",
    "revenue_by_category": "category",
    "customer_ltv": "ltv_segment",
}

def convert_to_category(dfs, config):
    for dataset, col in config.items():
        dfs[dataset][col] = dfs[dataset][col].astype("category")

convert_to_category(dfs, cols)

In [10]:
for key in dfs.keys():
    dfs[key].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype                    
---  ------           --------------  -----                    
 0   month            112 non-null    datetime64[ns, UTC+03:00]
 1   category         112 non-null    category                 
 2   monthly_revenue  112 non-null    float64                  
 3   payment_count    112 non-null    int64                    
dtypes: category(1), datetime64[ns, UTC+03:00](1), float64(1), int64(1)
memory usage: 3.5 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   weekday       168 non-null    category
 1   hour          168 non-null    int64   
 2   rental_count  168 non-null    int64   
dtypes: category(1), int64(2)
memory usage: 3.3 KB
<class 'pandas.core.frame.DataFrame'>
RangeI